In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [44]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers, output_size):
        super(SimpleNN, self).__init__()
        self.hidden=[]
        self.input = nn.Linear(input_size, hidden_size)
        for i in range(num_layers-2):
            self.hidden.append(nn.Linear(hidden_size, hidden_size))
        self.output = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.input(x)
        out = self.relu(out)
        for i in self.hidden:
            out = i(out)
            out = self.relu(out)
        out = self.output(out)
        return out


In [45]:
def initialize_population(population_size, num_genes):
    return np.random.randint(0, 2, size=(population_size, num_genes))

def decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range, num_hidden_layers_range, epochs_range):
    # ,num_hidden_layers,epochs
    learning_rate = learning_rate_range[0] + int(''.join(map(str, chromosome[:5])), 2) / 31.0 * (learning_rate_range[1] - learning_rate_range[0])
    num_hidden_neurons = num_hidden_neurons_range[0] + int(''.join(map(str, chromosome[5:10])), 2) / 31.0 * (num_hidden_neurons_range[1] - num_hidden_neurons_range[0])
    num_hidden_layers = num_hidden_layers_range[0]+ int(''.join(map(str, chromosome[10:12])), 2) /3 * (num_hidden_layers_range[1] - num_hidden_layers_range[0])
    epochs = epochs_range[0]+ int(''.join(map(str, chromosome[12:])), 2) /3 * (epochs_range[1] - epochs_range[0])
    return learning_rate, int(num_hidden_neurons),int(num_hidden_layers),int(epochs)

def fitness(learning_rate, epochs, num_input_neurons, num_hidden_neurons,num_hidden_layers,num_output_neurons, X_train, y_train, X_val, y_val):
    model = SimpleNN(input_size= num_input_neurons, hidden_size=num_hidden_neurons, num_layers=num_hidden_layers, output_size=num_output_neurons)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    num_epochs = epochs
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_val).sum().item() / len(y_val)

    return accuracy


In [46]:
def genetic_algorithm(population_size, num_genes, learning_rate_range, num_epochs_range, num_input_neurons,num_hidden_neurons_range,num_hidden_layers_range,num_output_neurons, X_train, y_train, X_val, y_val, num_generations):
    population = initialize_population(population_size, num_genes)
    for generation in range(num_generations):
        fitness_scores = []
        for chromosome in population:
            learning_rate, num_hidden_neurons, num_hidden_layers, epochs = decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range,num_hidden_layers_range,num_epochs_range)
            fitness_scores.append(fitness(learning_rate, epochs, num_input_neurons,num_hidden_neurons,num_hidden_layers,num_output_neurons, X_train, y_train, X_val, y_val))

        # parents based on fitness
        parents = population[np.argsort(fitness_scores)[-2:]]


        # Crossover
        crossover_point = np.random.randint(1, num_genes)
        children = []
        for i in range(population_size - 2):
            parent1 = parents[i % 2]
            parent2 = parents[(i + 1) % 2]
            child = []
            for j in range(num_genes):
                child.append([parent1[j], parent2[j]][np.random.randint(0,2)])

            # child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child = np.asarray(child)
            children.append(child)

        # Mutation
        mutation_rate = 0.1
        for i in range(population_size - 2):
            if np.random.rand() < mutation_rate:
                mutation_point = np.random.randint(num_genes)
                children[i][mutation_point] = 1 - children[i][mutation_point]
        best_chromosome = population[np.argmax(fitness_scores)]
        # Update population
        population = np.vstack((parents, np.array(children)))


    learning_rate, num_hidden_neurons, num_hidden_layers, epochs = decode_chromosome(best_chromosome, learning_rate_range, num_hidden_neurons_range,num_hidden_layers_range,num_epochs_range)
            
    return learning_rate, num_hidden_neurons, num_hidden_layers, epochs, max(fitness_scores)


In [47]:
iris = load_iris()
X = iris.data
y = iris.target
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [48]:
learning_rate_range = [0.001, 0.5]
num_hidden_neurons_range = [5, 100]
num_layers_range = [1,3]
num_epochs_range = [1,3]
in_size = 4
out_size = 3
best_learning_rate, best_num_hidden_neurons, best_num_hidden_layers, best_epochs, xyz = genetic_algorithm(population_size=75, num_genes=14,
                                                                learning_rate_range=learning_rate_range,
                                                                num_epochs_range=num_epochs_range,
                                                                num_input_neurons=in_size,
                                                                num_hidden_neurons_range=num_hidden_neurons_range,
                                                                num_hidden_layers_range=num_layers_range,
                                                                num_output_neurons = out_size,
                                                                X_train=X_train, y_train=y_train,
                                                                X_val=X_val, y_val=y_val,
                                                                num_generations=200)

    
print("Best learning rate:", best_learning_rate)
print("Best number of hidden neurons:", best_num_hidden_neurons)
print("Best number of hidden layers:", best_num_hidden_layers)
print("Best number of epochs:", best_epochs)
print("Best fitness score:", xyz)


Best learning rate: 0.3551290322580645
Best number of hidden neurons: 44
Best number of hidden layers: 3
Best number of epochs: 2
Best fitness score: 0.9666666666666667
